In [9]:
import numpy as np
from scipy.stats import multiscale_graphcorr
from sklearn.preprocessing import OneHotEncoder

In [22]:
def k_sample_test(X, y,score_func):
    one_hot_enc = OneHotEncoder() #onehotencoding
    temp =  one_hot_enc.fit_transform(y.reshape(-1,1))
    y = temp.toarray()
    if score_func == "mgc":
        test = multiscale_graphcorr(X, y) #mgc case
    else: 
        test = multiscale_graphcorr(X, y) #default is mgc as no other tests in scipy
    return(test.stat,test.pvalue)

In [44]:
#kBest fit_transform with k class object parameter provided as a function parameter,and score_func class object param as function param
def fit_transform(X,y,k,score_func):
    data = []
    scores = []
    features = X.shape[1]
    splits = np.log2(features)
    copy1 = X
    while (len(scores) < (features / splits)):
        beta = False
        X = copy1
        while(beta == False):
            copy2 = X
            X= X[:, np.random.permutation(X.shape[1])]
            X = np.array_split(X,2,axis=1)
            X1 = X[0]
            X2 = X[1]
            test1= k_sample_test(X1,y,score_func)
            test2 = k_sample_test(X2, y,score_func)
            if(test1[0] > test2[0]):
                X = X1
                X_stat = test1[0]
            else:
                X = X2
                X_stat = test2[0]
            if (X.shape[1]  == k):
                beta = True
            if (X.shape[1] < k):
                X = copy2
        data.append(X)
        scores.append(X_stat)
    sorted_index = np.argsort(scores)
    best = sorted_index[len(scores)-1]
    X_new = data[best]   
    return(X)
        
    
    

In [45]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)
print(X.shape)
X_new = fit_transform(X,y,1,"mgc")
print(X_new.shape)

(150, 4)
(150, 1)
